## Inicializacion

In [ ]:
!pip install gdown
!pip install deep-translator
!pip install imbalanced-learn
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install pandas

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import string
import re
import os
import numpy as np
import warnings
import gdown
from IPython.display import display
import time
import sys
from IPython.display import clear_output
from typing import List, Dict

[nltk_data] Downloading package stopwords to C:\Users\Franco-
[nltk_data]     SEC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    """
    TextPreprocessor es una clase de preprocesamiento de texto para limpiar datos textuales.
    Hereda de BaseEstimator y TransformerMixin de scikit-learn para integrarse en pipelines
    y garantizar compatibilidad con la API de scikit-learn.
    """
    
    def __init__(self):
        # Conjunto de stopwords en inglés que serán eliminadas del texto
        self.stop_words = set(stopwords.words('english'))
    
    def clean_text(self, text: str) -> str:
        """
        Limpia el texto de entrada realizando los siguientes pasos:
        1. Convierte el texto a minúsculas.
        2. Elimina la puntuación.
        3. Elimina las stopwords.
        
        Parámetros:
        text (str): Texto de entrada a limpiar.

        Retorna:
        str: Texto limpio.
        """
        # Convertir el texto a minúsculas
        text = text.lower()
        # Eliminar puntuación del texto
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Eliminar stopwords filtrando palabras en el texto
        text = ' '.join(word for word in text.split() if word not in self.stop_words)
        return text
    
    def fit(self, X: pd.Series, y=None) -> "TextPreprocessor":
        """
        Método de ajuste para cumplir con la API de scikit-learn.
        Este transformador no necesita aprender nada de los datos.
        
        Parámetros:
        X (pd.Series): Serie de pandas con los datos textuales de entrada.
        y: Parámetro opcional para compatibilidad; no se utiliza.

        Retorna:
        TextPreprocessor: Retorna la instancia actual del objeto.
        """
        return self
    
    def transform(self, X: pd.Series) -> pd.Series:
        """
        Aplica el proceso de limpieza de texto a los datos de entrada.
        
        Parámetros:
        X (pd.Series o str): Datos de entrada a transformar, ya sea una Serie de pandas o una cadena individual.
        
        Retorna:
        pd.Series o str: Datos limpiados, retornados como una Serie de pandas si la entrada fue una Serie,
                         o como una cadena individual si la entrada fue una cadena.
        
        Excepciones:
        ValueError: Si el tipo de entrada no es ni una Serie de pandas ni una cadena.
        """
        # Verificar si la entrada es una Serie de pandas
        if isinstance(X, pd.Series):
            return X.apply(self.clean_text)
        # Verificar si la entrada es una cadena individual
        elif isinstance(X, str):
            return self.clean_text(X)
        # Lanzar un error si el tipo de entrada es inválido
        else:
            raise ValueError("La entrada debe ser una cadena o una Serie de pandas.")

## Carga

In [22]:
def load_database() -> None:
    """
    Descarga varios archivos de datos desde Google Drive y los guarda en el directorio 'database'.
    Cada archivo se descarga mediante su ID de Google Drive específico y se guarda con un nombre predefinido.
    """
    
    # Diccionario que contiene el ID de cada archivo y su ruta de salida correspondiente
    files_to_download = {
        '1xkN6-OKnp8XOCjcRXFHyFAmKN6GODpji': 'database/bgg_database.csv',
        '1b4PUV-SRkUm7A_vLeRZMkA80rsABJjuV': 'database/IMDB-Movie-Data.csv',
        '1zJYm3kKzy1HQzta6aCmikTENgPOIrx28': 'database/libros.csv',
        '1dfduFDeHbIFoXhNj8FCV7yvuvwKn_rpl': 'database/negative-words.txt',
        '17jI37fqKDp9yzgAiwTIE3UmFvkL5XHO1': 'database/positive-words.txt'
    }

    # Descargar cada archivo usando su ID y guardarlo en la ubicación especificada
    for file_id, output_path in files_to_download.items():
        # Construir la URL de descarga usando el ID del archivo
        download_url = f'https://drive.google.com/uc?id={file_id}'
        # Descargar el archivo desde Google Drive y guardarlo en la ruta especificada
        gdown.download(download_url, output_path, quiet=True)

In [23]:
def load_books(developer_mode: bool = False) -> None:
    """
    Descarga información de libros populares del sitio Proyecto Gutenberg.
    Extrae el título, autor, enlace, resumen y temas de cada libro, y guarda los datos en un archivo CSV.

    Parámetros:
    developer_mode (bool): Modo desarrollador. Si es True, imprime información detallada de cada libro mientras se procesa.

    Salida:
    None: La función guarda los datos en un archivo CSV y no retorna ningún valor.
    """
    
    # URL de la página de libros más populares
    url: str = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"

    # Realizar solicitud a la página principal de libros
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Seleccionar enlaces de libros que contienen '/ebooks/' seguido de un número (ID de libro)
    libros = soup.select("li > a[href^='/ebooks/']")
    datos_libros: List[Dict[str, str]] = []  # Lista para almacenar la información de cada libro

    # Procesar cada enlace de libro
    for libro in libros:
        texto_completo: str = libro.get_text()
        enlace_anidado: str = libro['href']
        
        # Verificar que el enlace tenga un número después de "/ebooks/"
        if enlace_anidado.split('/ebooks/')[-1].isdigit():
            # Separar título y autor; si no hay autor, se asigna "Desconocido"
            titulo_y_autor: List[str] = texto_completo.split(" by ", 1)
            titulo: str = titulo_y_autor[0].strip()
            autor: str = titulo_y_autor[1].strip() if len(titulo_y_autor) > 1 else "Desconocido"
            
            # Construir URL completa para la página del libro
            url_libro: str = f"https://www.gutenberg.org{enlace_anidado}"
            
            # Realizar solicitud a la página individual del libro
            response_libro = requests.get(url_libro)
            response_libro.raise_for_status()
            soup_libro = BeautifulSoup(response_libro.text, 'html.parser')
            
            # Intentar extraer el resumen si está disponible
            summary: str = ""
            summary_row = soup_libro.find('th', text='Summary')
            if summary_row:
                summary_td = summary_row.find_next("td")
                if summary_td:
                    summary = summary_td.get_text(strip=True)
            
            # Extraer temas (subjects) del libro
            subjects: List[str] = []
            subject_rows = soup_libro.find_all('th', text='Subject')
            for subject_row in subject_rows:
                subject_td = subject_row.find_next("td")
                if subject_td:
                    subject_links = subject_td.find_all('a')
                    for link in subject_links:
                        subjects.append(link.get_text(strip=True))

            # Combinar los temas en una sola cadena
            subjects_combined: str = ", ".join(subjects)

            # Añadir la información del libro a la lista de datos
            datos_libros.append({
                "Título": titulo,
                "Autor": autor,
                "Enlace": url_libro,
                "Resumen": summary,
                "Subjects": subjects_combined
            })
            
            # Imprimir información detallada en modo desarrollador
            if developer_mode:
                print("Título:", titulo, "Autor:", autor, "URL:", url_libro, "Resumen:", summary, "Temas:", subjects_combined)

    # Crear un DataFrame y guardar los datos en un archivo CSV
    df_libros = pd.DataFrame(datos_libros)
    df_libros.to_csv('libros.csv', index=False)

In [25]:
def load_words(file_path: str) -> set[str]:
    """
    Carga palabras desde un archivo y las retorna como un conjunto de cadenas.
    
    Parámetros:
    file_path (str): La ruta del archivo de texto que contiene una palabra por línea.
    
    Retorna:
    Set[str]: Un conjunto de palabras (strings) obtenidas del archivo, sin espacios en blanco.
    """
    with open(file_path, 'r') as file:
        # Lee todas las líneas del archivo y elimina los espacios en blanco alrededor de cada palabra
        words = {line.strip() for line in file.readlines()}
    
    return words

In [26]:
def load_datasets_union() -> pd.DataFrame:
    """
    Carga y preprocesa tres conjuntos de datos (películas, juegos de mesa y libros),
    limpiando el texto de las columnas relevantes y combinándolos en un solo DataFrame.
    
    Realiza las siguientes acciones:
    1. Procesa y limpia los datos de películas.
    2. Procesa y limpia los datos de juegos de mesa.
    3. Procesa y limpia los datos de libros.
    4. Combina todos los datos en un solo DataFrame.

    Retorna:
    pd.DataFrame: Un DataFrame combinado que contiene los datos preprocesados.
    """
    # Instanciar el preprocesador de texto
    text_preprocessor = TextPreprocessor()

    # 1. Procesamiento de Películas
    peliculas_dataframe = pd.read_csv('database/IMDB-Movie-Data.csv')
    peliculas_dataframe['category'] = 'pelicula'
    peliculas_dataframe['text'] = (
        peliculas_dataframe['Title'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Genre'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Description'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Director'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Actors'].fillna('').apply(text_preprocessor.transform)
    )
    peliculas_dataframe.rename(columns={'Title': 'Titulo', 'Director': 'Autor', 'Description': 'Resumen', 'Genre': 'Subjects'}, inplace=True)

    # 2. Procesamiento de Juegos de Mesa
    juegos_mesa_dataframe = pd.read_csv('database/bgg_database.csv')
    juegos_mesa_dataframe['category'] = 'juego'
    juegos_mesa_dataframe['text'] = (
        juegos_mesa_dataframe['game_name'].fillna('').apply(text_preprocessor.transform) + " " +
        juegos_mesa_dataframe['description'].fillna('').apply(text_preprocessor.transform) + " " +
        juegos_mesa_dataframe['designers'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('') + " " +
        juegos_mesa_dataframe['mechanics'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('') + " " +
        juegos_mesa_dataframe['categories'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('')
    )
    juegos_mesa_dataframe.rename(columns={'game_name': 'Titulo','game_href': 'Enlace', 'designers': 'Autor', 'description': 'Resumen', 'mechanics': 'Subjects'}, inplace=True)

    # 3. Procesamiento de Libros
    libros_dataframe = pd.read_csv('database/libros.csv')
    libros_dataframe['category'] = 'libro'
    libros_dataframe['text'] = (
        libros_dataframe['Título'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Resumen'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Subjects'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Autor'].fillna('').apply(text_preprocessor.transform)
    )
    libros_dataframe.rename(columns={'Título': 'Titulo', 'Autor': 'Autor', 'Resumen': 'Resumen', 'Subjects': 'Subjects'}, inplace=True)

    # Concatenar los DataFrames de películas, juegos de mesa y libros en un solo DataFrame
    dataframe_bbdd = pd.concat(
        [peliculas_dataframe[['Titulo', 'Autor', 'Resumen', 'Subjects', 'category', 'text']],
         juegos_mesa_dataframe[['Titulo', 'Enlace','Autor', 'Resumen', 'Subjects', 'category', 'text']],
         libros_dataframe[['Titulo', 'Enlace','Autor', 'Resumen', 'Subjects', 'category', 'text']]
        ],
        ignore_index=True  # Para reiniciar el índice después de la concatenación
    )

    return dataframe_bbdd

In [27]:
def count_words(text: str, positive_words: set[str], negative_words: set[str]) -> pd.Series:
    """
    Cuenta las palabras positivas y negativas en un texto dado.

    Esta función cuenta cuántas palabras del texto pertenecen al conjunto de
    palabras positivas y cuántas al conjunto de palabras negativas.

    Parámetros:
    - text (str): El texto en el cual se realizarán las búsquedas de palabras.
    - positive_words (Set[str]): Un conjunto de palabras consideradas positivas.
    - negative_words (Set[str]): Un conjunto de palabras consideradas negativas.

    Retorna:
    - pd.Series: Una serie de pandas con dos valores, el conteo de palabras positivas
      en el texto y el conteo de palabras negativas en el texto, en ese orden.
    """
    # Contar las palabras positivas en el texto
    positive_count = sum(1 for word in text.split() if word in positive_words)
    
    # Contar las palabras negativas en el texto
    negative_count = sum(1 for word in text.split() if word in negative_words)
    
    # Retornar los resultados como una Serie de pandas
    return pd.Series([positive_count, negative_count], index=['Positive_Count', 'Negative_Count'])

## Modelos

In [28]:
def model_sentiments(developer_mode: bool) -> tuple[LogisticRegression, set[str], set[str]]:
    """
    Entrena un modelo de regresión logística para predecir sentimientos (positivo o negativo) basado en palabras clave positivas y negativas.

    La función carga un conjunto de palabras positivas y negativas desde archivos, las convierte en un DataFrame y genera
    características basadas en la frecuencia de aparición de palabras positivas y negativas en el texto. Luego entrena un modelo
    de regresión logística para clasificar las palabras como "feliz" (positivo) o "triste" (negativo).

    Parámetros:
    - developer_mode (bool): Un indicador para mostrar las métricas de evaluación del modelo (accuracy, matriz de confusión, etc.).

    Retorna:
    - model (LogisticRegression): El modelo entrenado de regresión logística.
    - positive_words (Set[str]): El conjunto de palabras positivas cargadas.
    - negative_words (Set[str]): El conjunto de palabras negativas cargadas.
    """
    
    # Cargar palabras positivas y negativas
    positive_words = load_words('database/positive-words.txt')
    negative_words = load_words('database/negative-words.txt')
    
    # Crear un conjunto de datos basado en las palabras
    data = {
        'text': [],
        'label': []
    }

    # Llenar el conjunto de datos con palabras positivas y negativas
    for word in positive_words:
        data['text'].append(word)
        data['label'].append('feliz')  # Etiqueta "feliz" para palabras positivas

    for word in negative_words:
        data['text'].append(word)
        data['label'].append('triste')  # Etiqueta "triste" para palabras negativas

    # Crear un DataFrame
    df = pd.DataFrame(data)

    # Crear características
    df[['positive_count', 'negative_count']] = df['text'].apply(lambda x: count_words(x, positive_words, negative_words))

    # Convertir etiquetas a números: "feliz" -> 1, "triste" -> 0
    df['label'] = df['label'].map({'feliz': 1, 'triste': 0})

    # Separar características y etiquetas
    X = df[['positive_count', 'negative_count']]  # Características
    y = df['label']  # Etiquetas

    # Dividir el conjunto de datos en entrenamiento y prueba (60% entrenamiento, 40% prueba)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Entrenar el modelo de regresión logística
    model = LogisticRegression()
    model.fit(X_train, y_train)

    if developer_mode:
        # Predecir en el conjunto de prueba
        y_pred = model.predict(X_test)

        # Evaluar el modelo
        accuracy = accuracy_score(y_test, y_pred)
        confusion = confusion_matrix(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f'Accuracy: {accuracy}')
        print('Confusion Matrix:')
        print(confusion)
        print('Classification Report:')
        print(report)
    
    # Retornar el modelo entrenado y los conjuntos de palabras positivas y negativas
    return model, positive_words, negative_words

In [29]:
def train_model(dataset: pd.DataFrame, developer_mode: bool) -> any:
    """
    Entrena un modelo de clasificación de texto utilizando regresión logística y TF-IDF para vectorización de texto.
    
    La función divide el conjunto de datos en entrenamiento y prueba, luego entrena un modelo con regresión logística y realiza 
    una validación cruzada para evaluar el rendimiento del modelo. Si 'developer_mode' es True, imprime las puntuaciones de la 
    validación cruzada y la puntuación media.
    
    Parámetros:
    - dataset (pd.DataFrame): Un DataFrame con columnas 'text' (texto) y 'category' (categorías de las clases).
    - developer_mode (bool): Un indicador para imprimir las métricas de la validación cruzada si es True.

    Retorna:
    - model (Pipeline): El modelo entrenado con regresión logística y vectorización TF-IDF.
    """
    
    # Separar el dataset en conjunto de entrenamiento y conjunto de prueba (80% - 20%)
    X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['category'], test_size=0.2, random_state=42)
    
    # Crear un pipeline con TF-IDF para la vectorización de texto y regresión logística
    model = make_pipeline(
        TfidfVectorizer(),  # Vectoriza el texto con TF-IDF
        LogisticRegression(C=1.0, max_iter=200, solver='liblinear')  # Modelo de regresión logística
    )
    
    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Realizar validación cruzada (5 pliegues)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # 5-fold cross-validation
    
    # Si 'developer_mode' es True, imprimir detalles de la validación cruzada
    if developer_mode:
        print(f"Cross-validation scores: {cv_scores}")
        print(f"Mean cross-validation score: {cv_scores.mean()}")
    
    return model

In [30]:
def model_recommender(dataset) -> any:
    """
    Esta función vectoriza los textos del dataset utilizando el algoritmo TF-IDF (Term Frequency-Inverse Document Frequency)
    para representar los textos como vectores numéricos.

    Parámetros:
    - dataset (pd.DataFrame): El DataFrame que contiene la columna 'text', que contiene los textos a vectorizar.

    Retorna:
    - tfidf_matrix (scipy.sparse.csr_matrix): Matriz dispersa de características vectorizadas del texto.
    - tfidf_vectorizer (TfidfVectorizer): El objeto del vectorizador que puede usarse para transformar más texto.
    """
    # Crear un objeto TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Ajustar y transformar los textos en una matriz TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['text'])
    
    return tfidf_matrix, tfidf_vectorizer

## Predicciones

In [31]:
def procesar_entrada_usuario(texto) -> str:
    """
    Procesa la entrada del usuario de la siguiente manera:
    1. Traduce el texto de español a inglés.
    2. Preprocesa el texto traducido para limpieza (ej. eliminación de stopwords, caracteres no deseados).

    Parámetros:
    - texto (str): Texto en español ingresado por el usuario.

    Retorna:
    - texto_procesado (str): Texto limpio y procesado en inglés.
    """
    # Traducir al inglés
    texto_traducido = GoogleTranslator(source='es', target='en').translate(texto)

    # Preprocesar el texto traducido (limpieza de texto)
    texto_procesado = TextPreprocessor().clean_text(texto_traducido)

    return texto_procesado

In [32]:
def recommend_item(prompt, model) -> str:
    """
    Esta función recomienda un artículo basado en el prompt del usuario.
    El modelo predictivo se usa para predecir la categoría del texto de entrada.
    
    Parámetros:
    - prompt (str): El texto de entrada del usuario.
    - model (modelo entrenado): El modelo que se usará para predecir la categoría.

    Retorna:
    - category (str): La categoría predicha por el modelo para el prompt ingresado.
    """
    # Predecir la categoría del prompt
    category = model.predict([prompt])[0]
    
    return category

In [33]:
def predict_sentiments(model, positive_words, negative_words, frase, developer_mode) -> str:
    """
    Esta función predice la emoción (feliz o triste) en función de la frase dada.
    La predicción se realiza contando las palabras positivas y negativas en la frase procesada 
    y utilizando un modelo previamente entrenado.

    Parámetros:
    - model (LogisticRegression o similar): El modelo entrenado para predecir la emoción.
    - positive_words (list): Lista de palabras positivas.
    - negative_words (list): Lista de palabras negativas.
    - frase (str): La entrada de texto a procesar y analizar.
    - developer_mode (bool): Indica si se deben imprimir detalles adicionales para depuración.

    Retorna:
    - emocion (str): La emoción predicha, que puede ser 'feliz' o 'triste'.
    """
    
    # Procesar la entrada de texto (traducción y limpieza)
    entrada_procesada = procesar_entrada_usuario(frase)
    
    if developer_mode:
        print("Texto procesado:", entrada_procesada)

    # Contar las palabras positivas y negativas en la entrada procesada
    entrada_counts = count_words(entrada_procesada, positive_words, negative_words)

    # Asegurarse de que la entrada esté en formato 2D para la predicción
    entrada_vectorizada = np.array([entrada_counts]).reshape(1, -1)

    # Obtener la predicción del modelo: 1 para 'feliz', 0 para 'triste'
    prediccion = model.predict(entrada_vectorizada)[0]

    # Determinar la emoción basada en la predicción (1 -> feliz, 0 -> triste)
    emocion = 'feliz' if prediccion == 1 else 'triste'
    
    return emocion

In [34]:
def predict_recomendation(prompt, category, model, dataframe_bbdd) -> tuple[str, str]:
    """
    Esta función genera recomendaciones basadas en un 'prompt' de entrada,
    filtrando las recomendaciones según la categoría dada, y luego calculando
    la similitud de coseno entre el 'prompt' y los elementos del conjunto de datos.

    Parámetros:
    - prompt (str): El texto de entrada para la recomendación.
    - category (str): La categoría de recomendaciones a considerar.
    - model (Pipeline): El modelo entrenado que incluye el vectorizador TF-IDF.
    - dataframe_bbdd (DataFrame): El conjunto de datos de recomendaciones, con columnas 'category', 'text', 'Titulo', 'Enlace'.
    
    Retorna:
    - Titulo (str): El título de la recomendación más similar al 'prompt'.
    - Enlace (str): El enlace relacionado con la recomendación.
    """
    
    # Filtrar el dataframe_bbdd para obtener solo las recomendaciones de la categoría dada.
    recommendations = dataframe_bbdd[dataframe_bbdd['category'] == category].copy().reset_index(drop=True)

    # Si no hay recomendaciones disponibles para la categoría, devolver un mensaje.
    if recommendations.empty:
        return "No hay recomendaciones disponibles", ""

    # Vectorizar el 'prompt' de recomendación utilizando el vectorizador TF-IDF del modelo.
    prompt_vector = model.named_steps['tfidfvectorizer'].transform([prompt])

    # Vectorizar el texto de las recomendaciones del dataset usando el vectorizador TF-IDF del modelo.
    dataset_vector = model.named_steps['tfidfvectorizer'].transform(recommendations['text'])

    # Calcular la similitud de coseno entre el 'prompt' y el texto de las recomendaciones del dataset.
    similarity_scores = cosine_similarity(prompt_vector, dataset_vector)

    # Obtener el índice del ítem más similar en el dataset basado en la similitud más alta.
    most_similar_idx = similarity_scores.argmax()

    # Obtener la recomendación correspondiente del dataframe_bbdd utilizando el índice de la similitud más alta.
    recommended_item = recommendations.iloc[most_similar_idx]

    # Devolver el título y el enlace de la recomendación más similar.
    return recommended_item['Titulo'], recommended_item['Enlace']

## Bot

In [35]:
def input_user():
    """
    Esta función solicita al usuario una entrada con dos partes: 
    un sentimiento y una recomendación, separados por una coma. 
    Si el usuario escribe "chau", la función termina la interacción.
    
    Retorna:
    - prompt_sentimiento (str): El texto procesado relacionado con el sentimiento.
    - prompt_recomendacion (str): El texto procesado relacionado con la recomendación, si existe.
    """
    
    # Solicitar la entrada del usuario
    prompt = input("¿Cómo estás hoy? ¿Qué quieres hacer? Puedes despedirte diciendo 'Chau': ")
    
    # Verificar si el usuario quiere terminar la interacción
    if prompt.lower() == "chau":
        return "exit", "exit"
    
    # Separar el prompt en sentimientos y recomendaciones usando la coma como delimitador
    prompts = prompt.split(",")
    
    # Limpiar y asignar los textos a sus respectivas variables
    sentiment_prompt = prompts[0].strip()  # El texto antes de la coma se considera el sentimiento
    recommendation_prompt = prompts[1].strip() if len(prompts) > 1 else ""  # El texto después de la coma se considera la recomendación, si existe

    # Traducir los prompts de español a inglés usando Google Translator
    translated_sentiment = GoogleTranslator(source='es', target='en').translate(sentiment_prompt)
    translated_recommendation = GoogleTranslator(source='es', target='en').translate(recommendation_prompt)

    # Procesar el texto para eliminar caracteres no deseados y hacer limpieza
    prompt_sentimiento = TextPreprocessor().clean_text(translated_sentiment)
    prompt_recomendacion = TextPreprocessor().clean_text(translated_recommendation)

    # Retornar los prompts procesados
    return prompt_sentimiento, prompt_recomendacion

In [36]:
def print_slow(text, delay=0.04):
    """
    Imprime un texto lentamente, mostrando un carácter a la vez con un retraso especificado.
    
    Parámetros:
    - text (str): El texto que se imprimirá lentamente.
    - delay (float, opcional): El tiempo de espera entre cada carácter en segundos. 
      Por defecto es 0.04 segundos.
    """
    
    # Recorrer cada carácter en el texto
    for char in text:
        sys.stdout.write(char)  # Escribir el carácter en la salida estándar
        sys.stdout.flush()  # Asegurarse de que el carácter se imprima inmediatamente
        time.sleep(delay)  # Esperar antes de imprimir el siguiente carácter
    
    # Imprimir una nueva línea al final
    print()

In [37]:
def main(developer_mode) -> None:
    """
    Función principal que gestiona el flujo de trabajo de la aplicación.
    Verifica la existencia de archivos necesarios, entrena el modelo y
    maneja la interacción con el usuario para predecir sentimientos y 
    hacer recomendaciones.

    Parámetros:
    - developer_mode (bool): Si está activado, muestra información detallada de depuración.
    """

    # Verificar la existencia de los archivos necesarios
    required_files = [
        'database/bgg_database.csv',
        'database/IMDB-Movie-Data.csv',
        'database/libros.csv',
        'positive-words.txt',
        'negative-words.txt'
    ]

    # Comprobar si los archivos existen, si no, crear y cargar según corresponda
    if not all(os.path.exists(file) for file in required_files):
        # Crear el directorio 'database' si no existe
        if not os.path.exists('database'):
            os.makedirs('database')

        # Si el archivo de libros no existe, cargarlo
        if not os.path.exists('database/libros.csv'):
            load_books()

        # Cargar las bases de datos necesarias
        load_database()

    # Entrenar el modelo de sentimientos
    model_sents, positive_words, negative_words = model_sentiments(developer_mode)

    # Cargar el dataframe de la base de datos combinada
    dataframe_bbdd = load_datasets_union()

    # Entrenar el modelo de recomendaciones
    recommendation_model = train_model(dataframe_bbdd, developer_mode)

    # Ciclo principal de interacción con el usuario
    while True:
        # Obtener la entrada del usuario (sentimiento y recomendación)
        prompt_sentimiento, prompt_recomendacion = input_user()

        # Salir si el usuario escribe 'chau'
        if prompt_sentimiento == "exit":
            break

        # Predecir sentimiento
        sentimiento = predict_sentiments(model_sents, positive_words, negative_words, prompt_sentimiento, developer_mode)

        # Predecir la categoría para la recomendación
        recommended_category = recommend_item(prompt_recomendacion, recommendation_model)

        if developer_mode:
            # Imprimir información detallada de la depuración
            print("Prompt sentimiento:", prompt_sentimiento)
            print("Prompt recomendación:", prompt_recomendacion)
            print("Categoría recomendada:", recommended_category)

        # Obtener la recomendación final (título y enlace)
        recommended_title, recommended_link = predict_recomendation(prompt_recomendacion, recommended_category, recommendation_model, dataframe_bbdd)

        # Mostrar los resultados con efectos visuales
        print_slow(f"Entiendo que hoy estás {sentimiento}")
        print_slow(f"Mi recomendación es: {recommended_title} y el enlace es: {recommended_link} \n")

Hoy fue un buen dia, quiero ver una pelicula de criminales intergalacticos
Tenemos un dia increible con amigos, queremos un juego de mesa de aventuras y ciencia ficcion
Hoy fue un dia horrible, quiero leer un libro de biologia
Hoy me encuentro mal, quiero ver una pelicula de comedia
Hoy discuti con un amigo, me gustaria algo divertido para estar solo

In [38]:
def chatbot() -> None:
    """
    Función principal que interactúa con el usuario, configura el modo de desarrollador,
    muestra un mensaje de bienvenida con instrucciones, y luego llama a la función principal
    que gestiona la predicción de sentimientos y recomendaciones.
    """

    # Activar el modo desarrollador según la respuesta del usuario
    developer_mode = False
    warnings.filterwarnings("ignore")  # Ignorar advertencias durante la ejecución

    # Preguntar al usuario si desea activar el modo desarrollador
    if input("¿Desea activar el modo desarrollador? (s/n): \n") == "s":
        developer_mode = True

    # Limpiar la salida de consola antes de mostrar los mensajes de bienvenida
    clear_output(wait=False)

    # Mostrar mensajes de bienvenida y reglas de interacción
    print("-------------------------------------------------------")
    print_slow("Hola, mi nombre es ChatLPJ, antes de empezar necesito darte algunas reglas")
    print_slow('1) No entiendo negativos, por ejemplo "No quiero" o "No me siento bien"')
    print_slow('2) Necesito que tus respuestas siempre sean "(como te sentis),(que buscas)" para entenderte mejor!')
    print_slow('3) A veces mis respuestas no son las mejores, te pido disculpas, mis creadores tuvieron algunos problemas cuando me diseñaron')
    print_slow("Ahora te pido un minuto para prepararme")
    print("-------------------------------------------------------")

    # Llamar a la función principal que maneja la predicción y recomendaciones
    main(developer_mode)

    # Despedirse del usuario al finalizar
    print("Nos vemos 👋")


## Chat

In [40]:
chatbot()

-------------------------------------------------------
Hola, mi nombre es ChatLPJ, antes de empezar necesito darte algunas reglas
1) No entiendo negativos, por ejemplo "No quiero" o "No me siento bien"
2) Necesito que tus respuestas siempre sean "(como te sentis),(que buscas)" para entenderte mejor!
3) A veces mis respuestas no son las mejores, te pido disculpas, mis creadores tuvieron algunos problemas cuando me diseñaron
Ahora te pido un minuto para prepararme
-------------------------------------------------------
Accuracy: 0.9996318114874816
Confusion Matrix:
[[1879    0]
 [   1  836]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1879
           1       1.00      1.00      1.00       837

    accuracy                           1.00      2716
   macro avg       1.00      1.00      1.00      2716
weighted avg       1.00      1.00      1.00      2716

Cross-validation scores: [0.99125 0.9925  0.99375 0.